In [1]:
from hdbscan import HDBSCAN
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rosbags.typesys import get_types_from_msg, register_types
from rosbags.highlevel import AnyReader
from pathlib import Path
import tqdm



RADARSPOKE_MSG = """
uint32[32] time
float32[32] bearing
uint16[32] spoke_index
uint16[32] range
float32[32] latitude
float32[32] longitude
uint8[16384] spoke
uint8[32] doppler_mode
uint8[16384] doppler_spoke
"""
TOPIC = "/marv/nav/radar/spoke"
MSG_TYPE = "marv_msgs/msg/RadarSpoke"
register_types(get_types_from_msg(RADARSPOKE_MSG, MSG_TYPE))

out_folder = "/Users/hugodrak/Documents/chalmers/1_kandarb_EENX16/CASE/cpp_playground/cv2_images/old_log"


#df = pd.DataFrame(columns=["time", 'x', 'y'])

LOGPATH = "/Users/hugodrak/Documents/chalmers/1_kandarb_EENX16/logs_ockero_08_05_23/rosbag2_2023_05_08-15_50_22/"
N = 10000
ti = 0
dbscan = HDBSCAN(min_cluster_size=20)#30
prev_rng = None
rot_c = 0
frame_i = 0
detections = 0

In [7]:

ri = 750 * np.array(range(0, 512))
sdf = pd.DataFrame(columns=["time", 'bear'])
with AnyReader([Path(LOGPATH)]) as reader:
	connections = [x for x in reader.connections if x.topic == TOPIC]
	for connection, timestamp, rawdata in tqdm.tqdm(reader.messages(connections=connections)):
		msg = reader.deserialize(rawdata, connection.msgtype)
		#t_str = pd.to_datetime(timestamp, unit='ns').strftime('%H:%M:%S')
		bears = np.array(msg.bearing)
		ts = np.array(msg.time)
		sdf = pd.concat([sdf, pd.DataFrame({"time": ts, "bear": bears})], ignore_index=True)

sdf



0it [00:00, ?it/s]/var/folders/g8/4rhybrsx49v63_kdkt3rz91r0000gn/T/ipykernel_6358/1153524150.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sdf = pd.concat([sdf, pd.DataFrame({"time": ts, "bear": bears})], ignore_index=True)
6202it [00:19, 316.37it/s]


,time,bear
0,59089,13.535156
1,59089,13.710938
2,59089,13.886719
3,59089,14.062500
4,59089,8.613281
...,...,...
198459,59705,305.156250
198460,59705,305.332031
198461,59705,305.507812
198462,59705,305.683594


In [8]:
sdf_32nd = sdf.iloc[::32]
sdf_32nd


,time,bear
0,59089,13.535156
32,59089,13.535156
64,59089,24.785156
96,59089,30.410156
128,59089,30.410156
...,...,...
198304,59704,227.285156
198336,59704,289.160156
198368,59705,294.785156
198400,59705,294.785156


In [43]:
from rosbags.highlevel import AnyReader
from pathlib import Path
import os
import pandas as pd
LOGPATH = "/Users/hugodrak/Documents/chalmers/1_kandarb_EENX16/CASE/cpp_playground/5-2/rosbag2_2024_02_20-11_24_00"

def print_meta(reader):
	swe_tz = 3600000000000
	dur = str(round(reader.duration*10**(-9), 2))
	st = pd.to_datetime(reader.start_time+swe_tz, unit='ns').strftime('%H:%M:%S')
	et = pd.to_datetime(reader.end_time+swe_tz, unit='ns').strftime('%H:%M:%S')
	mc = str(reader.message_count)
	topics = sorted([f"{('('+str(v.msgcount)+')').rjust(8)} {k}" for k,v in reader.topics.items()])
	w = max([len(dur), len(et), len(mc), len(st)]+ [len(t)+3 for t in topics])
	print("╔"+"═"*(w+2)+"╗")
	print("║"+" "*((w-6)//2)+"LOG DATA"+" "*((w-5)//2)+"║")
	print("╠"+"═"*(w+2)+"╣")
	print("║"+"Duration: "+dur + " s"+" "*(w-len(dur)-10)+"║")
	print("║"+"Start time: "+st+" "*(w-len(st)-10)+"║")
	print("║"+"End time: "+et+" "*(w-len(et)-8)+"║")
	print("║"+"Message count: "+mc+" "*(w-len(mc)-13)+"║")
	print("║"+"Topics: "+" "*(w-6)+"║")
	for t in topics:
		print("║"+"   "+t+" "*(w-len(t)-1)+"║")
	print("╚"+"═"*(w+2)+"╝")





# 35 duration
# 36 end_time

# 39 message_count

# 44 start_time
# 45 topics

with AnyReader([Path(LOGPATH)]) as reader:
	print_meta(reader)


100
╔═══════════════════════════════════════════════╗
║                   LOG DATA                    ║
╠═══════════════════════════════════════════════╣
║Duration: 46.7 s                               ║
║Start time: 11:24:56                           ║
║End time: 11:25:43                             ║
║Message count: 3592                            ║
║Topics:                                        ║
║        (0) /marv/nav/sbg_acceleration         ║
║        (0) /marv/nav/sbg_angular_velocity     ║
║        (0) /marv/nav/sbg_current_pos          ║
║        (0) /marv/nav/sbg_ekf_status           ║
║        (0) /marv/nav/sbg_pose                 ║
║        (0) /marv/nav/sbg_ref_pos              ║
║        (0) /marv/nav/sbg_velocity             ║
║        (0) /marv/nav/sbg_velocity_magnitude   ║
║        (0) /marv/sys/ctrl/cmd_steering        ║
║        (0) /marv/sys/log/log1_ncu             ║
║        (0) /marv/sys/log/log1_tcu             ║
║        (0) /marv/sys/status/heartbeat_ncu   